This notebook calculates various statistics for networks

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import numpy as np


ModuleNotFoundError: No module named 'networkx'

In [ ]:
# Activate testing
TEST = False
PLOTS = True
PATH = "./saved-48"
NAME = "base"

SEGPLOTS = False
# SEGPATH = ["./saved-b13", "./saved-b15", "./saved-48", "./saved-b19"]
# SEGPATH = ["./saved-4grp", "./saved-mm", "./saved-48"]
SEGPATH = ["./saved-4type", "./saved-cross", "./saved-seg", "./saved-48"]
SEGNAME = "type"

GET_SEEDS = False
SEEDPATH = "./saved-4type"


# Global constants
# COLOURS = ['firebrick', 'tomato', 'peru','darkgreen', 'mediumseagreen', 'teal']
# standard
COLOURS = ['firebrick', 'tomato','darkgreen', 'mediumseagreen']
# 4 type
# COLOURS = ['firebrick','darkgreen','navy','purple','tomato','mediumseagreen','cornflowerblue','violet']
# 4 grp
# COLOURS = ['firebrick','tomato','darkgreen','mediumseagreen','navy','cornflowerblue','purple','violet']


==============================================================================================================================

In [ ]:
def create_colour_map(G, colours):

    if (len(colours) < len(G.graph['population_breakdown'] * G.graph['population_breakdown'][0])):
        raise ValueError("Missing colours")
    
    colour_map = []

    for node in G.nodes:
        grp = G.nodes[node]['group']
        tp = G.nodes[node]['type']
        colour_map.append(colours[grp * len(G.graph['population_breakdown'][0]) + tp])
    
    return colour_map

In [ ]:
# Returns a matrix of size n x n showing inter- and intra group connections
def stratify_connections(G):
    
    groups = len(G.graph['population_breakdown']) # number of social groups
    
    group_connections = np.zeros((groups, groups), dtype=int)
                                  
    for edge in G.edges:
        node1 = G.nodes[edge[0]]
        node2 = G.nodes[edge[1]]
        
        # populate upper triangle first
        lower_edge = min(node1['group'], node2['group'])
        upper_edge = max(node1['group'], node2['group'])
        group_connections[lower_edge, upper_edge] += 1
    
    # copy upper triangle into lower triangle
    group_connections = group_connections + np.transpose(group_connections) - np.diag(np.diag(group_connections))
    
    return group_connections
    

In [ ]:
def get_connectivity(G):
    connection_matrix = stratify_connections(G)
    
    intra_connections = np.trace(connection_matrix)
    inter_connections = np.sum(np.triu(connection_matrix,1))
    all_connections = intra_connections + inter_connections
    
    all_possible_connections = 0.5 * len(G) * (len(G) - 1)
    
    return all_connections / all_possible_connections
    

In [ ]:
def get_interconnect_ratio(connection_matrix):
    # get number of connections by typology
    intra_connections = np.trace(connection_matrix)
    inter_connections = np.sum(np.triu(connection_matrix,1))
    all_connections = intra_connections + inter_connections
    
    if (all_connections == 0):
        return 0
    else:
        return inter_connections / all_connections
    

In [ ]:
def calculate_freeman_segregation(G):
    
    # get matrix of all connections
    connection_matrix = stratify_connections(G)
    
    proportion_inter = get_interconnect_ratio(connection_matrix)
    
    sum_of_squared_n = 0
    sum_of_n = 0
    for group in G.graph['population_breakdown']:
        sum_of_squared_n += sum(group) ** 2
        sum_of_n += sum(group) 
    
    freeman = 1 - (proportion_inter * sum_of_n * (sum_of_n - 1)) / (sum_of_n ** 2 - sum_of_squared_n)
    
    return freeman

In [ ]:
def calculate_incremental_segregation(G_IC, G_C):
    # get matrix of all connections
    connection_matrix_IC = stratify_connections(G_IC)
    connection_matrix_C = stratify_connections(G_C)
    
    proportion_inter_IC = get_interconnect_ratio(connection_matrix_IC)
    proportion_inter_C = get_interconnect_ratio(connection_matrix_C)
    
    # not defined if complete information case is empty
    if (proportion_inter_C == 0):
        return np.nan
    return (proportion_inter_C - proportion_inter_IC) / proportion_inter_C


    

In [ ]:
def calculate_avg_shortest_path(G):
    avg_shortest_path = []
    for C in (G.subgraph(c).copy() for c in nx.connected_components(G)):
        avg_shortest_path.append(nx.average_shortest_path_length(C))
        
    return avg_shortest_path

In [ ]:
def calculate_information(G):
    return np.sum(G.graph['information']) / (len(G) ** 2)


In [ ]:
def calculate_avg_var(metric):
    average = []
    variance = []
    for m in metric:
        avg = sum(m) / len(m)
        var = sum((x-avg)**2 for x in m) / len(m)
        average.append(avg)
        variance.append(var)
    return average, variance

In [ ]:
def get_metrics(G):
    
    freeman = calculate_freeman_segregation(G)
    
    cliques = nx.graph_number_of_cliques(G)

    if (len(calculate_avg_shortest_path(G)) > 0):
        avg_shortest_path = sum(calculate_avg_shortest_path(G)) / len(calculate_avg_shortest_path(G))
    
    if (len(nx.degree_centrality(G)) > 0):
        dc = nx.degree_centrality(G).values()
        avg_degree_centrality = sum(dc) / len(dc)
        var_degree_centrality = sum((x-avg_degree_centrality)**2 for x in dc) / len(dc)
    
    information = calculate_information(G)
        
    return freeman, cliques, avg_shortest_path, avg_degree_centrality, var_degree_centrality, information
        
    

In [ ]:
def display_metrics(G):
    freeman, cliques, avg_shortest_path, avg_degree_centrality, var_degree_centrality, information = get_metrics(G)
    
    print("========================================================================")
    print("This graph has %s nodes with population: %s" %(len(G), G.graph['population_breakdown']))
    print("The parameters are, delta=%s, c_l=%s, c_h=%s" %(G.graph['delta'], G.graph['cost_low'], G.graph['cost_high']))
    print("Convergence: %s after %s periods" %(G.graph['converged'], G.graph['epochs']))
    print("Freeman's SI: %s" %round(freeman,4))
    print("Cliques: %s" %cliques)
    print("Avg shortest path: %s" %round(avg_shortest_path,4))
    print("Avg degree centrality: %s" %round(avg_degree_centrality,4))
    print("Var degree centrality: %s" %round(var_degree_centrality,4))
    print("Information: %s" %round(information,4))
 
    print("Adjacency matrix:")
    print(nx.adjacency_matrix(G).todense())
    print("========================================================================")


==============================================================================================================================

Script to demonstrate how pickles are read and metrics are displayed

In [ ]:
# Read and display a pickled graph
read = False
pickle_path = f"{PATH}/IC-0-ch1.0-belief6.gpickle"
if (read):
    G = nx.read_gpickle(pickle_path)
    colour_map = create_colour_map(G, COLOURS)
    nx.draw(G, node_color=colour_map, with_labels=True)
    plt.show()
    display_metrics(G)


In [ ]:
# Read and display a pickled graph
read = False
pickle_path = f"{PATH}/C-0-ch1.25-belief6.gpickle"
if (read):
    G = nx.read_gpickle(pickle_path)
    colour_map = create_colour_map(G, COLOURS)
    nx.draw(G, node_color=colour_map, with_labels=True)
    plt.show()
    display_metrics(G)

==============================================================================================================================

The following functions are used to read batches of graphs and compute various statistics using the above functions

In [ ]:
def get_convergence(root, belief, case, files, increments):
    convergence_IC = []
    convergence_C = []
    convergence_R = []
    
    for cost in increments:
        # temporary arrays for each increment

        conv_IC = []
        conv_C = []
        conv_R = []

        for iteration in range(0,files):
            path_IC = root + "/IC-%s-%s%s-belief%s.gpickle" %(iteration, case, cost, belief)
            path_C = root + "/C-%s-%s%s-belief%s.gpickle" %(iteration, case, cost, belief)
            path_R = root + "/IC-%s-%s%s-belief%s.gpickle" %(iteration, case, cost, 0)
            G_IC = nx.read_gpickle(path_IC)
            G_C = nx.read_gpickle(path_C)
            G_R = nx.read_gpickle(path_R)

            if (G_IC.graph['converged'] and G_C.graph['converged'] and G_R.graph['converged']):  
                
                # epochs to convergence as tuples (IC, C)
                c_IC = G_IC.graph['epochs']
                c_C = G_C.graph['epochs']
                c_R = G_R.graph['epochs']
                conv_IC.append(c_IC)
                conv_C.append(c_C)
                conv_R.append(c_R)
                
        convergence_IC.append(conv_IC)
        convergence_C.append(conv_C)
        convergence_R.append(conv_R)

    return convergence_IC, convergence_C, convergence_R


In [ ]:
def check_attribute(option, root, belief, case, files, increments):
    
    IC = np.zeros((len(increments), files), dtype=int)
    C = np.zeros((len(increments), files), dtype=int)
    R = np.zeros((len(increments), files), dtype=int)
    
    for index, cost in enumerate(increments):
        for iteration in range(0,files):
            path_IC = root + "/IC-%s-%s%s-belief%s.gpickle" %(iteration, case, cost, belief)
            path_C = root + "/C-%s-%s%s-belief%s.gpickle" %(iteration, case, cost, belief)
            path_R = root + "/IC-%s-%s%s-belief%s.gpickle" %(iteration, case, cost, 0)
            G_IC = nx.read_gpickle(path_IC)
            G_C = nx.read_gpickle(path_C)
            G_R = nx.read_gpickle(path_R)
            
            if option == 0:
                IC[index, iteration] = int(G_IC.graph['converged'])
                C[index, iteration] = int(G_C.graph['converged'])
                R[index, iteration] = int(G_R.graph['converged'])

            if option == 1:
                IC[index, iteration] = int(G_IC.graph['epochs'])
                C[index, iteration] = int(G_C.graph['epochs'])
                R[index, iteration] = int(G_R.graph['epochs'])
    
    return IC, C, R

In [ ]:
# opens up all files specified in the directory passed in as root and first calculates statistics for each run before averaging across all runs
def calculate_stats(root, belief, case, files, increments, base=True):

    # arrays to be returned
    freeman = []
    incr_segregation_C = []
    incr_segregation_R = []
#     cliques_IC = []
#     cliques_C = []
#     avg_shortest_path_IC = []
#     avg_shortest_path_C = []
    avg_degree_centrality_IC = []
    avg_degree_centrality_C = []
    avg_degree_centrality_R = []
#     var_degree_centrality_IC = []
#     var_degree_centrality_C = []
    information_IC = []
    information_R = []
    
    for cost in increments:
        # temporary arrays for each increment
        free = []
        incr_seg_C = []
        incr_seg_R = []
#         cliq_IC = []
#         cliq_C = []
#         asp_IC = []
#         asp_C = []
        adc_IC = []
        adc_C = []
        adc_R = []
#         vdc_IC = []
#         vdc_C = []        
        info_IC = []
        info_R = []
        
        for iteration in range(0,files):
            path_IC = root + "/IC-%s-%s%s-belief%s.gpickle" %(iteration, case, cost, belief)
            G_IC = nx.read_gpickle(path_IC)
            
            if base:
                path_C = root + "/C-%s-%s%s-belief%s.gpickle" %(iteration, case, cost, belief)
                path_R = root + "/IC-%s-%s%s-belief%s.gpickle" %(iteration, case, cost, 0)
                G_C = nx.read_gpickle(path_C)
                G_R = nx.read_gpickle(path_R)

            # freeman and incremental segregation
            f = calculate_freeman_segregation(G_IC)
            free.append(f)
            
            if base:
                i_s_C = calculate_incremental_segregation(G_IC, G_C)
                i_s_R = calculate_incremental_segregation(G_IC, G_R)                
                incr_seg_C.append(i_s_C)
                incr_seg_R.append(i_s_R)

            # cliques
#                 cq_IC = nx.graph_number_of_cliques(G_IC)
#                 cq_C = nx.graph_number_of_cliques(G_C)

#                 cliq_IC.append(cq_IC)   
#                 cliq_C.append(cq_C)   

            # avg shortest path
#                 asp_components_IC = calculate_avg_shortest_path(G_IC)
#                 asp_components_C = calculate_avg_shortest_path(G_C)

#                 if (len(asp_components_IC) > 0 and len(asp_components_C) > 0):
#                     a_IC = sum(asp_components_IC) / len(asp_components_IC)
#                     a_C = sum(asp_components_C) / len(asp_components_C)
#                     asp_IC.append(a_IC)
#                     asp_C.append(a_C)

            # degree centrality
            dc_IC = nx.degree_centrality(G_IC)
            avg_dc_IC = sum(dc_IC.values())/len(dc_IC.values())

            if base:
                dc_C = nx.degree_centrality(G_C)
                dc_R = nx.degree_centrality(G_R)
                avg_dc_C = sum(dc_C.values())/len(dc_C.values())
                avg_dc_R = sum(dc_R.values())/len(dc_R.values())

#                 var_dc_IC = sum((x-avg_dc_IC)**2 for x in dc_IC.values()) / len(dc_IC.values())
#                 var_dc_C = sum((x-avg_dc_C)**2 for x in dc_C.values()) / len(dc_C.values())

            adc_IC.append(avg_dc_IC)
            if base:
                adc_C.append(avg_dc_C)
                adc_R.append(avg_dc_R)
#                 vdc_IC.append(var_dc_IC)
#                 vdc_C.append(var_dc_C)

            # information
            inf_IC = calculate_information(G_IC)
            info_IC.append(inf_IC)
            if base:
                inf_R = calculate_information(G_R)
                info_R.append(inf_R)
                
        freeman.append(free)
        if base:
            incr_segregation_C.append(incr_seg_C)
            incr_segregation_R.append(incr_seg_R)
#         cliques_IC.append(cliq_IC)
#         cliques_C.append(cliq_C)
#         avg_shortest_path_IC.append(asp_IC)
#         avg_shortest_path_C.append(asp_C)
        avg_degree_centrality_IC.append(adc_IC)
        if base:
            avg_degree_centrality_C.append(adc_C)
            avg_degree_centrality_R.append(adc_R)
#         var_degree_centrality_IC.append(vdc_IC)
#         var_degree_centrality_C.append(vdc_C)
        information_IC.append(info_IC)
        if base:
            information_R.append(info_R)
    
    return freeman, incr_segregation_C, incr_segregation_R, \
            avg_degree_centrality_IC, avg_degree_centrality_C, avg_degree_centrality_R, \
            information_IC, information_R 

In [ ]:
def make_plot(i, axs, label, increments, \
            freeman, incr_segregation_C, incr_segregation_R, \
            avg_degree_centrality_IC, avg_degree_centrality_C, avg_degree_centrality_R, \
            information_IC, information_R):
    
    freeman_avg, freeman_var = calculate_avg_var(freeman)
    incr_segregation_C_avg, incr_segregation_C_var = calculate_avg_var(incr_segregation_C)
    incr_segregation_R_avg, incr_segregation_R_var = calculate_avg_var(incr_segregation_R)
    
    smallfont = 14
    largefont = 18
    lw = 4
    ms = 8
    labelpad_y = -5
    labelpad_x = 5
    
    main_line = 'dashed'
    sub_line = 'solid'
    
    ###
    ax1 = axs[0, i]
    line1, = ax1.plot(increments, freeman_avg, color='crimson', marker='o', linestyle=main_line, linewidth=lw, markersize=ms)
    line2, = ax1.plot(increments, incr_segregation_C_avg, color='mediumseagreen', linestyle=sub_line, linewidth=lw, markersize=ms)
    line3, = ax1.plot(increments, incr_segregation_R_avg, color='royalblue', linestyle=sub_line, linewidth=lw, markersize=ms)

    ax1.tick_params(labelsize = smallfont)
#     ax1.set_xlabel(label, fontsize=largefont)
    if i == 0:
        ax1.set_ylabel('Segregation \n Index', fontsize=largefont,labelpad=labelpad_y)
    if i == 1:
        ax1.legend((line1, line2, line3), ('Freeman\'s SI', 'ISI complete', 'ISI rational'), fontsize=smallfont, loc='upper left')
    ax1.set_xlim(increments[0] - 0.01, increments[-1] + 0.01)
    ax1.set_xticks(np.arange(increments[0], increments[-1] + 0.2, 0.2))
    ax1.set_ylim(-0.2, 1.1)
    ax1.set_yticks(np.arange(-0.2, 1.2, 0.2))
    
    ###
    info_IC, info_IC_var = calculate_avg_var(information_IC)
    info_R, info_R_var = calculate_avg_var(information_R)

    ax2 = axs[1, i]
    line1, = ax2.plot(increments, info_IC, color='crimson', marker='o', linestyle=main_line, linewidth=lw, markersize=ms)
    line2, = ax2.plot(increments, info_R, color='royalblue', linestyle=sub_line, linewidth=lw, markersize=ms)

    ax2.tick_params(labelsize = smallfont)
#     ax2.set_xlabel(label, fontsize=largefont)
    if i == 0:
        ax2.set_ylabel('Proportion of \n Discovery', fontsize=largefont, labelpad=labelpad_y)
    if i == 1:
        ax2.legend((line1, line2), ('Biased', 'Rational'), fontsize=smallfont, loc='lower left')
    ax2.set_xlim(increments[0] - 0.01, increments[-1] + 0.01)
    ax2.set_xticks(np.arange(increments[0], increments[-1] + 0.2, 0.2))
    ax2.set_ylim(-0.01,0.71)

    ###
#     cliq_IC, cliq_IC_var = calculate_avg_var(cliques_IC)
#     cliq_C, cliq_C_var = calculate_avg_var(cliques_C)
    
#     ax3 = axs[1,1]
#     line1, = ax3.plot(increments, cliq_IC, color='royalblue', marker='o', linestyle='dashed', linewidth=lw, markersize=ms)
#     line2, = ax3.plot(increments, cliq_C, color='royalblue', linestyle='solid', linewidth=lw, markersize=ms)
#     ax3.tick_params(labelsize = smallfont)
#     ax3.set_xlabel(label, fontsize=largefont)
#     ax3.set_ylabel('Number of Cliques', fontsize=largefont)
#     ax3.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
#     ax3.legend((line1, line2), ('Incomplete Info', 'Complete Info'), fontsize=smallfont)

#     asp_IC, asp_IC_var = calculate_avg_var(avg_shortest_path_IC)
#     asp_C, asp_C_var = calculate_avg_var(avg_shortest_path_C)
    
#     ax4 = axs[1,2]
#     line1, = ax4.plot(increments, asp_IC, color='peru', marker='o', linestyle='dashed', linewidth=lw, markersize=ms)
#     line2, = ax4.plot(increments, asp_C, color='peru', linestyle='solid', linewidth=lw, markersize=ms)
#     ax4.tick_params(labelsize = smallfont)
#     ax4.set_xlabel(label, fontsize=largefont)
#     ax4.set_ylabel('Average Shortest Path', fontsize=largefont)
#     ax4.legend((line1, line2), ('Incomplete Info', 'Complete Info'), fontsize=smallfont)
#     ax4.set_ylim(bottom=0)

    
    ###
    adc_IC, adc_IC_var = calculate_avg_var(avg_degree_centrality_IC)
    adc_C, adc_C_var = calculate_avg_var(avg_degree_centrality_C)
    adc_R, adc_R_var = calculate_avg_var(avg_degree_centrality_R)
    
    dc_color = 'teal'
    ax5 = axs[2, i]
    line1, = ax5.plot(increments, adc_IC, color='crimson', marker='o', linestyle=main_line, linewidth=lw, markersize=ms)
    line2, = ax5.plot(increments, adc_C, color='mediumseagreen', linestyle=sub_line, linewidth=lw, markersize=ms)
    line3, = ax5.plot(increments, adc_R, color='royalblue', linestyle=sub_line, linewidth=lw, markersize=ms)
    ax5.tick_params(labelsize = smallfont)
    ax5.set_xlabel(label, fontsize=largefont, labelpad=labelpad_x)
    if i == 0:
        ax5.set_ylabel('Avg Degree \n Centrality', fontsize=largefont, labelpad=labelpad_y)
        ax5.legend((line1, line2, line3), ('Biased', 'Complete', 'Rational'), fontsize=smallfont)
    ax5.set_xlim(increments[0] - 0.01, increments[-1] + 0.01)
    ax5.set_xticks(np.arange(increments[0], increments[-1] + 0.2, 0.2))
    ax5.set_ylim(-0.01, 0.61)

    #     ax5.set_ylim(top=0.7)

    
#     vdc_IC, vdc_IC_var = calculate_avg_var(var_degree_centrality_IC)
#     vdc_C, vdc_C_var = calculate_avg_var(var_degree_centrality_C)
    
#     ax6 = axs[1,1]
#     line1, = ax6.plot(increments, vdc_IC, color=dc_color, marker='o', linestyle='dashed', linewidth=lw, markersize=ms)
#     line2, = ax6.plot(increments, vdc_C, color=dc_color, linestyle='solid', linewidth=lw, markersize=ms)
#     ax6.tick_params(labelsize = smallfont)
#     ax6.set_xlabel(label, fontsize=largefont)
#     ax6.set_ylabel('Variance Degree Centrality', fontsize=largefont)
#     ax6.set_ylim(top=0.012)
#     ax6.set_ylim(bottom=0)
#     ax6.legend((line1, line2), ('Incomplete Info', 'Complete Info'), fontsize=smallfont)

In [ ]:
def make_seg_plot(i, axs, label, increments, \
            freeman, avg_degree_centrality, information):
    
    freeman_avg = []
    for f in freeman:
        avg, var = calculate_avg_var(f)
        freeman_avg.append(avg)
        
    smallfont = 14
    largefont = 18
    lw = 4
    ms = 8
    labelpad_y = -5
    labelpad_x = 5
    
    # colors = ['greenyellow','mediumseagreen','crimson','royalblue']
    # linestyles = ['solid','solid','dashed','solid']
    # markers = [None, None, 'o', None]
    # labels = [r'$\beta = 3$', r'$\beta = 5$', r'$\beta = 7$', r'$\beta = 9$']
    # labels = ['4 Groups', 'Dominant', 'Base']

    colors = ['greenyellow','mediumseagreen','royalblue','crimson']
    linestyles = ['solid','solid','solid','dashed']
    markers = [None, None, None, 'o']
    labels = ['4 Types', 'Dominant', 'Correlated', 'Base']

    
    ###
    lines = []
    ax1 = axs[0, i]
    for case, f in enumerate(freeman_avg):
        line, = ax1.plot(increments, f, color=colors[case], marker=markers[case], linestyle=linestyles[case], linewidth=lw, markersize=ms)
        lines.append(line) 
#     line1, = ax1.plot(increments, freeman_avg[0], color='greenyellow', linestyle='solid', linewidth=lw, markersize=ms)
#     line2, = ax1.plot(increments, freeman_avg[1], color='mediumseagreen', linestyle='solid', linewidth=lw, markersize=ms)
#     line3, = ax1.plot(increments, freeman_avg[2], color='crimson', marker='o', linestyle='dashed', linewidth=lw, markersize=ms)
#     line4, = ax1.plot(increments, freeman_avg[3], color='royalblue', linestyle='solid', linewidth=lw, markersize=ms)

    ax1.tick_params(labelsize = smallfont)
#     ax1.set_xlabel(label, fontsize=largefont)
    if i == 0:
        ax1.set_ylabel('Freeman\'s SI', fontsize=largefont,labelpad=labelpad_y)
    if i == 0:
        ax1.legend(lines, labels , fontsize=smallfont-4, loc='lower right')
    ax1.set_xlim(increments[0] - 0.01, increments[-1] + 0.01)
    ax1.set_xticks(np.arange(increments[0], increments[-1] + 0.2, 0.2))
    ax1.set_ylim(-0.2, 1.1)
    ax1.set_yticks(np.arange(-0.2, 1.2, 0.2))

    ###
    info_avg = []
    for info in information:
        avg, var = calculate_avg_var(info)
        info_avg.append(avg)
        
    lines = []
    
    ax2 = axs[1, i]
    for case, info in enumerate(info_avg):
        line, = ax2.plot(increments, info, color=colors[case], marker=markers[case], linestyle=linestyles[case], linewidth=lw, markersize=ms)
        lines.append(line)
#     line1, = ax2.plot(increments, info_avg[0], color='greenyellow', linestyle='solid', linewidth=lw, markersize=ms)
#     line2, = ax2.plot(increments, info_avg[1], color='mediumseagreen',linestyle='solid', linewidth=lw, markersize=ms)
#     line3, = ax2.plot(increments, info_avg[2], color='crimson', marker='o', linestyle='dashed', linewidth=lw, markersize=ms)
#     line4, = ax2.plot(increments, info_avg[3], color='royalblue', linestyle='solid', linewidth=lw, markersize=ms)

    ax2.tick_params(labelsize = smallfont)
#     ax2.set_xlabel(label, fontsize=largefont)
    if i == 0:
        ax2.set_ylabel('Proportion of \n Discovery', fontsize=largefont, labelpad=labelpad_y)
#     ax2.legend((line1, line2, line3, line4), ('beta(1,3)', 'beta(1,5)', 'beta(1,7)', 'beta(1,9)'), fontsize=smallfont)
    ax2.set_xlim(increments[0] - 0.01, increments[-1] + 0.01)
    ax2.set_xticks(np.arange(increments[0], increments[-1] + 0.2, 0.2))
    ax2.set_ylim(-0.01,0.71)
    
    ###
    adc_avg = []
    for adc in avg_degree_centrality:
        avg, var = calculate_avg_var(adc)
        adc_avg.append(avg)
    
    lines = []
    
    ax5 = axs[2, i]
    for case, adc in enumerate(adc_avg):
        line, = ax5.plot(increments, adc, color=colors[case], marker=markers[case], linestyle=linestyles[case], linewidth=lw, markersize=ms)
        lines.append(line)
#     line1, = ax5.plot(increments, adc_avg[0], color='greenyellow', linestyle='solid', linewidth=lw, markersize=ms)
#     line2, = ax5.plot(increments, adc_avg[1], color='mediumseagreen', linestyle='solid', linewidth=lw, markersize=ms)
#     line3, = ax5.plot(increments, adc_avg[2], color='crimson', marker='o', linestyle='dashed', linewidth=lw, markersize=ms)
#     line4, = ax5.plot(increments, adc_avg[3], color='royalblue', linestyle='solid', linewidth=lw, markersize=ms)
    ax5.tick_params(labelsize = smallfont)
    ax5.set_xlabel(label, fontsize=largefont, labelpad=labelpad_x)
    if i == 0:
        ax5.set_ylabel('Avg Degree \n Centrality', fontsize=largefont, labelpad=labelpad_y)
#     ax5.legend((line1, line2, line3, line4), ('beta(1,3)', 'beta(1,5)', 'beta(1,7)', 'beta(1,9)'), fontsize=smallfont)
    ax5.set_xlim(increments[0] - 0.01, increments[-1] + 0.01)
    ax5.set_xticks(np.arange(increments[0], increments[-1] + 0.2, 0.2))
    ax5.set_ylim(-0.01, 0.61)


In [ ]:
def make_convergence_plot(axs, labels, increments, convergences):
    
    titles = ['(a)', '(b)', '(c)']
    colors = ['tomato', 'royalblue', 'peru']
    
    smallfont = 20
    largefont = 24
    lw = 4
    ms = 8
    
    for col in range(len(axs[0])):
        for row in range(len(axs)):
            
            conv_IC, conv_IC_var = calculate_avg_var(convergences[(col * len(axs) + row) * 2])
            conv_C, conv_C_var = calculate_avg_var(convergences[(col * len(axs) + row) * 2 + 1])
            ax = axs[row,col]
            line1, = ax.plot(increments[(col * len(axs) + row)], conv_IC, color=colors[col], marker='o', linestyle='dashed', linewidth=lw, markersize=ms)
            line2, = ax.plot(increments[(col * len(axs) + row)], conv_C, color=colors[col], marker='o', linestyle='solid', linewidth=lw, markersize=ms)
            ax.tick_params(labelsize = smallfont)
            ax.set_xlabel(labels[col * row + row], fontsize=largefont)
            ax.set_ylabel('Periods to Convergence', fontsize=largefont)
            ax.legend((line1, line2), ('Incomplete Info', 'Complete Info'), fontsize=smallfont)
            if(row==1):
                ax.set_title(titles[col], y=-0.4, fontsize=largefont)

    

==============================================================================================================================

The script below first calculates the statistics in batches and then uses make_plot functions to generate the plots and save it in the current directory
Note that cases should be saved in different files ending in unique digits!

In [ ]:
# check convergence
case = 'cl'
if (case == 'cl'):
    increments = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
else:
    increments = [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.15, 1.2, 1.25, 1.3, 1.35, 1.4, 1.45, 1.5]
files = 10
belief = 0
option = 1
# IC, C = check_attribute(option, PATH, belief, case, files, increments)
# print(IC)
# print(C)


In [ ]:
# producing all plots
if (PLOTS):
    root = PATH
    belief = 6    
    files = 30
    
    # 12 width x 9 height
    fig, axs = plt.subplots(3, 2, figsize = (9,9))

    for i,c in enumerate(['cl','ch']):
        case = c
        if (case == 'cl'):
            label = 'Intra-type cost ($c_L$)'
            increments = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
        else:
            label = 'Inter-type cost ($c_H$)'
            increments = [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.15, 1.2, 1.25, 1.3, 1.35, 1.4, 1.45, 1.5]

        # get arrays of size len(increments) containing the averages for each statistic
        freeman, incr_segregation_C, incr_segregation_R, \
                    avg_degree_centrality_IC, avg_degree_centrality_C, avg_degree_centrality_R, \
                    information_IC, information_R \
                    = calculate_stats(root, belief, case, files, increments)

        fig.subplots_adjust(wspace=0.25, hspace=0.15)
        make_plot(i, axs, label, increments, \
                    freeman, incr_segregation_C, incr_segregation_R, \
                    avg_degree_centrality_IC, avg_degree_centrality_C, avg_degree_centrality_R, \
                    information_IC, information_R)
        fig.align_ylabels(axs)
    
    plt.savefig(f'{NAME}.svg', format='svg', bbox_inches='tight')
    plt.close()



In [ ]:
# producing all plots
if SEGPLOTS:
    root = SEGPATH
    belief = 6    
    files = 30
    
    # 12 width x 9 height
    fig, axs = plt.subplots(3, 2, figsize = (9,9))

    for i,c in enumerate(['cl','ch']):
        case = c
        if (case == 'cl'):
            label = 'Intra-type cost ($c_L$)'
            increments = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
        else:
            label = 'Inter-type cost ($c_H$)'
            increments = [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.15, 1.2, 1.25, 1.3, 1.35, 1.4, 1.45, 1.5]

        # get arrays of size len(increments) containing the averages for each statistic
        
        f = []
        adc = []
        info = []
        
        for r in root:
            freeman, incr_segregation_C, incr_segregation_R, \
                        avg_degree_centrality_IC, avg_degree_centrality_C, avg_degree_centrality_R, \
                        information_IC, information_R \
                        = calculate_stats(r, belief, case, files, increments, base=False)
            f.append(freeman)
            adc.append(avg_degree_centrality_IC)
            info.append(information_IC)
            
        fig.subplots_adjust(wspace=0.25, hspace=0.15)
        make_seg_plot(i, axs, label, increments, \
                    f, adc, info)
        fig.align_ylabels(axs)
    
    plt.savefig(f'{SEGNAME}.svg', format='svg', bbox_inches='tight')
    plt.close()

In [ ]:
if GET_SEEDS:
    root = SEEDPATH
    belief = 6
    files = 30
    
    seeds_IC = []
    seeds_C = []
    seeds_R = []

    for i,c in enumerate(['cl','ch']):
        case = c
        if (case == 'cl'):
            label = 'Intra-type cost ($c_L$)'
            increments = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
        else:
            label = 'Inter-type cost ($c_H$)'
            increments = [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.15, 1.2, 1.25, 1.3, 1.35, 1.4, 1.45, 1.5]

        for cost in increments:
            for iteration in range(0,files):
                path_IC = root + "/IC-%s-%s%s-belief%s.gpickle" %(iteration, case, cost, belief)
                G_IC = nx.read_gpickle(path_IC)
                seeds_IC.append(G_IC.graph['seed'])

                path_C = root + "/C-%s-%s%s-belief%s.gpickle" %(iteration, case, cost, belief)
                G_C = nx.read_gpickle(path_C)
                seeds_C.append(G_C.graph['seed'])

                path_R = root + "/IC-%s-%s%s-belief%s.gpickle" %(iteration, case, cost, 0)
                G_R = nx.read_gpickle(path_R)
                seeds_R.append(G_R.graph['seed'])

    print(np.all(seeds_IC == seeds_C))
    print(np.all(seeds_IC == seeds_R))
    
    seeds = np.asarray(seeds_IC, dtype=int)
    np.savetxt(f"{root}/seeds.csv", seeds, delimiter=",")    

==============================================================================================================================

A few unit tests verifying statistics are calculated as expected

In [ ]:
import unittest

class Test(unittest.TestCase):

    def setup_test_graph(self):
        G = nx.Graph(population_breakdown = [[2],[3]])
        G.add_node(1, group=0)
        G.add_node(2, group=0)
        G.add_node(3, group=1)
        G.add_node(4, group=1)
        G.add_node(5, group=1)
        
        G.add_edge(1,2)
        G.add_edge(3,4)
        G.add_edge(1,3)
        G.add_edge(1,5)
        
        return G
    

    def test_connections_stratified(self): 
        G = self.setup_test_graph()
        
        strats = stratify_connections(G)
        expect = np.asarray([[1,2],[2,1]])
        
        self.assertTrue(np.all(expect == strats))
        
    def test_connection_ratio(self):
        G = self.setup_test_graph()
        strats = stratify_connections(G)
        
        ratio = get_interconnect_ratio(strats)
        
        self.assertTrue(ratio == 2/4)
        
    def test_freeman(self):
        G = self.setup_test_graph()
        freeman = calculate_freeman_segregation(G)
        pi = (2 * 2 * 3) / (5 * (5-1))
        ratio = get_interconnect_ratio(stratify_connections(G))
    
        self.assertTrue(freeman == 1 - ratio / pi)

    

In [ ]:
if(TEST):
    unittest.main(argv=[''], verbosity=2, exit=False)